In [1]:
os.chdir("../..")

In [2]:
import torch, torch.nn as nn
import pytorch_lightning as pl
import os
from models.ranked_transformer import HsqcRankedTransformer
from datasets.hsqc_folder_dataset import FolderDataModule

In [6]:
path = "/data/smart4.5/summer_runs_pretrain/hsqc_pretrain_08_10_2022_00:35_epochs=200_bs=64/checkpoints/epoch=18-step=29697.ckpt"
model = HsqcRankedTransformer.load_from_checkpoint(path).cuda()
my_dir = f"/workspace/smart4.5/tempdata/hyun_fp_data/hsqc_ms_pairs"
dm = FolderDataModule(dir=my_dir, do_hyun_fp=True, input_src=["HSQC"], batch_size=16)
dm.setup("fit")
val_dl = dm.val_dataloader()

In [7]:
for idx, batch in enumerate(val_dl):
    data, label = batch
    data, label = data.cuda(), label.cuda()
    out = model.forward(data)
    print(data.size(), label.size())
    print(out.size())
    break

torch.Size([16, 49, 3]) torch.Size([16, 6144])
torch.Size([16, 6144])
